# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
7/16/2019	A systems approach to assessing environmental and economic effects of food loss and waste interventions in the United States	Science of the Total Environment	Open access!	"Muth, M. K., C. Birney, A. Cu&eacute;llar, S. M. Finn, M. Freeman, J. N. Galloway, I. Gee, J. A. Gephart, K. Jones, L. Low, E. Meyer, Q. D. Read, T. Smith, K. A. Weitz, and S. Zoubek. 2019. A systems approach to assessing environmental and economic effects of food loss and waste interventions in the United States. Science of the Total Environment 685:1240-1254. DOI: 10.1016/j.scitotenv.2019.06.230."	muth-et-al-2019	https://www.sciencedirect.com/science/article/pii/S0048969719328037
2/27/2019	Towards connecting biodiversity and geodiversity across scales with satellite remote sensing	Global Ecology and Biogeography	Open access!	"Zarnetske, P. L., Q. D. Read, S. Record, K. Gaddis, S. Pau, M. Hobi, S. L. Malone, J. K. Costanza, K. M. Dahlin, A. Latimer, 

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [7]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2019-07-16,A systems approach to assessing environmental ...,Science of the Total Environment,Open access!,"Muth, M. K., C. Birney, A. Cu&eacute;llar, S. ...",muth-et-al-2019,https://www.sciencedirect.com/science/article/...
1,2019-02-27,Towards connecting biodiversity and geodiversi...,Global Ecology and Biogeography,Open access!,"Zarnetske, P. L., Q. D. Read, S. Record, K. Ga...",zarnetske-et-al-2019,https://onlinelibrary.wiley.com/doi/full/10.11...
2,2019-03-12,Fungal colonization of plant roots is resistan...,Ecosphere,Open access!,"Henning, J. A., Q. D. Read, N. J. Sanders, and...",henning-et-al-2019,https://esajournals.onlinelibrary.wiley.com/do...
3,2018-10-20,"Size, niches, and the latitudinal diversity gr...",Teaching Issues and Experiments in Ecology,Open access!,"Grady, J. M., Q. D. Read, S. Record, P. L. Zar...",grady-et-al-2018,http://tiee.esa.org/vol/v14/issues/figure_sets...
4,2018-10-01,Among-species overlap in rodent body size dist...,Ecography,Open access!,"Read, Q. D., J. M. Grady, P. L. Zarnetske, S. ...",read-et-al-2018-ecography,https://onlinelibrary.wiley.com/doi/10.1111/ec...
5,2018-01-24,Tropical bird species have less variable body ...,Biology Letters,NaN,"Read, Q. D., B. Baiser, J. M. Grady, P. L. Zar...",read-et-al-2018-biology-letters,https://royalsocietypublishing.org/doi/full/10...
6,2018-03-06,Aboveground resilience to species loss but bel...,Journal of Plant Ecology,NaN,"Read, Q. D., J. A. Henning, A. T. Classen, and...",read-et-al-2018-j-plant-ecology,https://academic.oup.com/jpe/article/11/3/351/...
7,2018-05-01,Short-term responses to warming vary between n...,Oecologia,NaN,"Welshofer, K. B., P. L. Zarnetske, N. K. Lany,...",welshofer-et-al-2018,https://link.springer.com/article/10.1007/s004...
8,2017-09-01,Intraspecific variation in traits reduces abil...,Journal of Vegetation Science,NaN,"Read, Q. D., J. A. Henning, and N. J. Sanders....",read-et-al-2017-j-veg-sci,https://onlinelibrary.wiley.com/doi/full/10.11...
9,2017-07-01,Consistently inconsistent drivers of patterns ...,Ecology,Open access! First author is an undergraduate ...,"Hendershot, J. N., Q. D. Read, J. A. Henning, ...",hendershot-et-al-2017,https://esajournals.onlinelibrary.wiley.com/do...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [8]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [9]:
# Some test code to see what the filename would be
# Looks like excel messed it up.
str(publications['pub_date'])

'0     2019-07-16\n1     2019-02-27\n2     2019-03-12\n3     2018-10-20\n4     2018-10-01\n5     2018-01-24\n6     2018-03-06\n7     2018-05-01\n8     2017-09-01\n9     2017-07-01\n10    2017-12-29\n11    2016-07-01\n12    2016-07-01\n13    2016-08-01\n14    2015-09-01\n15    2014-02-01\n16    2013-12-07\n17    2013-11-12\n18    2012-02-29\nName: pub_date, dtype: object'

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [10]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = str(item.pub_date)[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    ##md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [1]:
!ls ../_publications/

2012-02-29-clark-et-al-2012.md
2013-11-12-vannuland-et-al-2013.md
2013-12-07-gorman-et-al-2013.md
2014-02-01-read-et-al-2014-functional-ecology.md
2015-09-01-schussler-et-al-2015.md
2016-07-01-read-et-al-2016-oikos.md
2016-07-01-vannuland-et-al-2016.md
2016-08-01-yoon-and-read-2016.md
2017-07-01-hendershot-et-al-2017.md
2017-09-01-read-et-al-2017-j-veg-sci.md
2017-12-29-butler-et-al-2017.md
2018-01-24-read-et-al-2018-biology-letters.md
2018-03-06-read-et-al-2018-j-plant-ecology.md
2018-05-01-welshofer-et-al-2018.md
2018-10-01-read-et-al-2018-ecography.md
2018-10-20-grady-et-al-2018.md
2019-02-27-zarnetske-et-al-2019.md
2019-03-12-henning-et-al-2019.md
2019-07-16-muth-et-al-2019.md


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---
This paper is about the number 1. The number 2 is left for future work.

[Download paper here](http://academicpages.github.io/files/paper1.pdf)

Recommended citation: Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).